In [40]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

def get_html(url, params=None):
    response = requests.get(url=url, params=params)
    html = None

    if response.status_code == 200 and (params != None or (params == None and url == response.url)):
        html = BeautifulSoup(response.text, "lxml")
        print("* URL호출 성공 >> ", response.url)
    else:
        print("* request 실패(혹은 안함)! >> ", str(response.status_code))
        print(response.url)

    return html

url = "http://api.visitkorea.or.kr/guide/inforArea.do"
url_params = {"langtype" : "KOR",
              "arrange" : "A",
              "pageNo" : "1",
              "mode" : "listOk",
              "contenttypeid" : "12",
              "areacode" : "39"
             }

pageNo = 1

while True:
    url_params["pageNo"] = pageNo
    list_html = get_html(url, params=url_params)
    
    item_list = list_html.select("ul.galleryList li a")
    
    if len(item_list) > 0:
        print("# 진행페이지 >> ", pageNo, end="\n")
        for item in list_html.select("ul.galleryList li a"):
            detail_html = get_html("http://api.visitkorea.or.kr" + item.get("href"))

            result_xml1 = ET.fromstring(detail_html.select("#common .debugCon textarea")[0].text) # 공통정보
            result_xml2 = ET.fromstring(detail_html.select("#intro .debugCon textarea")[0].text) # TO-DO : 소개정보

            need_data_tags = ["title", "addr1", "zipcode", "tel", "mapx", "mapy", "overview"]

            for tag in need_data_tags:
                try:
                    result_xml1.find("./body/items/item/" + tag).text
                    print(tag + " >> " + result_xml1.find("./body/items/item/" + tag).text)
                except:
                    continue
                    
        pageNo += 1
    else:
        break
        
print("\n\n##### 프로그램 끝 #####")
                

* URL호출 성공 >>  http://api.visitkorea.or.kr/guide/inforArea.do?langtype=KOR&arrange=A&pageNo=26&mode=listOk&contenttypeid=12&areacode=39
# 진행페이지 >>  26
* URL호출 성공 >>  http://api.visitkorea.or.kr/guide/tourDetail.do?contentId=126469&langtype=KOR&typeid=12&oper=area&burl=&contentTypeId=12&areaCode=39&sigunguCode=&cat1=&cat2=&cat3=&listYN=Y&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&arrange=A&numOfRows=12&pageNo=26
title >> 한라산 영실
addr1 >> 제주특별자치도 서귀포시 영실로 246
zipcode >> 63554
mapx >> 126.4803110551
mapy >> 33.3402306397
overview >> 제주 한라산은 자연휴식년제를 실시 중인 돈내코 코스를 제외한 다섯 개의 탐방코스가 있다. 관음사, 어리목, 영실, 성판악을 기점으로 하는 코스와 어리목 부근의 기생화산인 어승생악 코스가 있다. 한라산 최고의 가을 단풍코스로는 단연 영실코스를 들 수 있다. 한라산 서측 99번 국도를 따라가면 어리목 입구를 지나 1100고지 휴게소를 넘어서 약 4km쯤 내려가면 영실입구 삼거리에 도착한다. 삼거리에서 매표소까지는 약 3km 거리. 매표소에서 영실휴게소까지는 다시 30~40분 정도 걸어 올라가야 한다. 영실 탐방코스(영실∼윗세오름)는 3.7km로 대략 1시간 40분 정도 소요된다. 영실휴게소를 출발하여 1km 남짓 비교적 완만한 길로 힘들이지 않고, 주변 수목과 영실 계곡 단풍을 즐기며 쉬엄쉬엄 오를 수 있다.<br /><br />본격적으로 들쭉날쭉한 기암괴석이 시야에 들어오기 시작하는 오르막길로 접어들면 가파른 산길이 능선을 따라

* URL호출 성공 >>  http://api.visitkorea.or.kr/guide/tourDetail.do?contentId=129619&langtype=KOR&typeid=12&oper=area&burl=&contentTypeId=12&areaCode=39&sigunguCode=&cat1=&cat2=&cat3=&listYN=Y&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&arrange=A&numOfRows=12&pageNo=26
title >> 협재굴(한림공원)
addr1 >> 제주특별자치도 제주시 한림읍 한림로 300
zipcode >> 63011
mapx >> 126.2381504642
mapy >> 33.3882881753
overview >> 협재굴은 황금굴, 소천굴, 쌍용굴, 만장굴과 더불어 제주도의 대표적인 용암동굴이다. 동굴의 길이는 약 200m, 너비 10m, 높이는 5m 정도의 규모이며, 250십만년 전 한라산 일대의 화산이 폭발하면서 생성되었다. 용암동굴이자 석회동굴의 특징이 복합된 2차원적인 동굴이라는 점이 특이하며, 그 일대가 모래와 조개껍질이 섞여있는 패사층으로 되어 있다. 동굴 내부에는 천장에서 뻗어 나온 석종과 마치 바닥으로부터 솟구쳐 나온 듯이 보이는 석순 등이 신비로운 광경을 연출하며, 석종과 석순이 만나 하나의 기둥을 이루는 종유석이 기괴하면서 아름답다. 동굴 벽면에는 석회분이 덮여있어 마치 하나의 거대한벽화가 새겨져 있는 듯 웅장한 모습을 보이고 있다. 동굴 내부의 온도는 연중내내 17~18℃를 유지하여 한여름의 이색적인 피서지로 각광을 받고 있고, 한 겨울에는 따뜻한 온도로 추위를 피할 수 있어 각광받고 있다. <br> 협재굴은 천연기념물 제 236호로 지정되어 있고, 페루의 돌소금동굴, 유고의 해중석회동굴과 더불에 세계 제3대 불가사의 동굴로 꼽힐만큼 유명한 곳이다. 협재굴을 빠져나오면 바로 옆에 쌍용굴이 자리하고 있어 두 동굴을 함께 돌아볼 수 있다.<br>
* URL호출 성공 >>  h

* URL호출 성공 >>  http://api.visitkorea.or.kr/guide/tourDetail.do?contentId=1890857&langtype=KOR&typeid=12&oper=area&burl=&contentTypeId=12&areaCode=39&sigunguCode=&cat1=&cat2=&cat3=&listYN=Y&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&arrange=A&numOfRows=12&pageNo=27
title >> 흙붉은오름
addr1 >> 제주특별자치도 서귀포시 남원읍 영실로
zipcode >> 63607
mapx >> 126.5616918017
mapy >> 33.3881016415
overview >> 한라산국립공원내에 제주시 아라동과 조천읍 교래리의 경계에 위치한 오름으로, 한라산 백록담을 중심으로 동부 산록상에서는 가장 높고,  한라산 정상에 가장 가까운 오름이다.<br />전체적인 형태는 동쪽으로 벌어진 말굽형 화구를 이루고, 화구 방향에 속칭 숫오름이라 불리는 돌오름이 마주해 있다. 화구의 잔디로 된 사면에는 시로미, 그 아래로 구상나무 등 고산식물이 자라고 있으며 오름 등성마루에서는 흰진달래 자생목이 발견된 바 있다.<br />돌오름(숫오름)과 비교하여 옥문형이라 일컬어지는 말굽형화구 깊숙한 곳에는 샘이 있는데, 이 샘은 기생화산 화구내의 샘으로서는 가장 높은 고도에 위치해 있는 샘이다.<br />※ 오름명의 유래오름의 흙이 매우 붉음으로 마을에서는 예전부터 흙붉은오름이라 부르고 있으며 한자로는 그 뜻을 빌어 土赤岳(토적악)이라 표기하고 있다.<br>
* URL호출 성공 >>  http://api.visitkorea.or.kr/guide/tourDetail.do?contentId=1861656&langtype=KOR&typeid=12&oper=area&burl=&contentTypeId=12&areaCode=39&sigunguCode=&cat1=&cat2=&cat3=&listYN=